# Imports

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

# Load Data

In [2]:
df = pd.read_csv("../datasets/BreadBasket_DMS_DT.csv")

# Sort Data by Transaction Order

In [3]:
transactions=[]
for item in df['Transaction'].unique():
    itemList = list(set(df[df['Transaction']==item]['Item']))
    transactions.append(itemList)

transactions[0:10]

[['Bread'],
 ['Scandinavian'],
 ['Cookies', 'Jam', 'Hot chocolate'],
 ['Muffin'],
 ['Pastry', 'Coffee', 'Bread'],
 ['Muffin', 'Pastry', 'Medialuna'],
 ['Tea', 'Pastry', 'Coffee', 'Medialuna'],
 ['Pastry', 'Bread'],
 ['Muffin', 'Bread'],
 ['Scandinavian', 'Medialuna']]

In [4]:
te = TransactionEncoder()
encodedData = te.fit(transactions).transform(transactions)
basketData = pd.DataFrame(encodedData, columns=te.columns_)
basketData

,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9526,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9527,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
9528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9529,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
